In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import random
from copy import deepcopy
import gc
import inspect
import os
from datetime import datetime
import json

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# 原始模型和分词器的初始化
origin_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to('cuda')
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"


In [3]:
src_text = "The concentration of these metals in crops varies depending on soil composition, and can be exacerbated by the extensive use of metal-containing fertilizers and persistent industrial pollution."
encoded_input = tokenizer(src_text, return_tensors="pt").to('cuda')

In [ ]:
generated_tokens = origin_model.generate(
        **encoded_input, 
        forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
    )
translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
translated_text = translated_text.replace(" ", "")
print(translated_text)

In [12]:
lm_head = origin_model.lm_head

In [13]:
print(lm_head)

Linear(in_features=1024, out_features=250054, bias=False)


In [11]:
print(generated_tokens[0])

tensor([     2, 250025,      6,   5001,      6, 107461,   6376,      6, 186005,
             6,    514,      6,     43,      6,  79138,   2621,      6, 208238,
             6, 177699,      6,     43,      6,  57372,      6,      4,      6,
         18941,      6,  12370,      6,  22566,      6,   2229,      6,  16005,
             6, 107461,      6,     43,      6,  64998,   8794,      6,    264,
             6,  23862,      6,     43,      6,  22065,      6,  34328,  56584,
             6,   1147,      6,  30966,   1988,      6,     30,      2],
       device='cuda:0')


In [4]:
print(origin_model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNor